## BERT Fine-tuning

In [1]:
! which python

/scratch-data/wu1396/condaenv/py37/bin/python


In [2]:
!pwd

/scratch-data/wu1396/complex-order_oct_16_2021/TEXT_CLASSIFICATION/Transformer/Experiment_BERT


In [3]:
!nvidia-smi

Sat Oct 23 19:59:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:18:00.0 Off |                  N/A |
| 32%   49C    P2    52W / 250W |      1MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# Imports

In [5]:
import torch

### Models

In [6]:
# from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForSequenceClassification
from myBERT import MyBertForSequenceClassification

In [7]:
# Fine-tuned Models

# model_name = "bhadresh-savani/distilbert-base-uncased-emotion"
# model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# model_name = "aychang/distilbert-base-cased-trec-coarse"

# model_name = "textattack/bert-base-uncased-SST-2"
# model_name = "aychang/bert-base-cased-trec-coarse"
# model_name = "textattack/bert-base-uncased-ag-news"
# model_name = "bhadresh-savani/bert-base-uncased-emotion"

# model_name = "textattack/roberta-base-SST-2"
# model_name = "bhadresh-savani/roberta-base-emotion"

# pt_model = AutoModelForSequenceClassification.from_pretrained(model_name, cache_dir="./cache/model").to("cuda:0")
# tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="./cache/model")

In [8]:
# Pre-trained Models
model_name = "bert-base-uncased"

model = MyBertForSequenceClassification.from_pretrained(model_name, num_labels=2, cache_dir="./cache/model").to("cuda:0")
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="./cache/model")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing MyBertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing MyBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MyBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MyBertForSequenceClassification were not initialized from the model check

In [9]:
model.bert.embeddings.position_embeddings.weight

Parameter containing:
tensor([[ 1.7505e-02, -2.5631e-02, -3.6642e-02,  ...,  3.3437e-05,
          6.8312e-04,  1.5441e-02],
        [ 7.7580e-03,  2.2613e-03, -1.9444e-02,  ...,  2.8910e-02,
          2.9753e-02, -5.3247e-03],
        [-1.1287e-02, -1.9644e-03, -1.1573e-02,  ...,  1.4908e-02,
          1.8741e-02, -7.3140e-03],
        ...,
        [ 1.7418e-02,  3.4903e-03, -9.5621e-03,  ...,  2.9599e-03,
          4.3435e-04, -2.6949e-02],
        [ 2.1687e-02, -6.0216e-03,  1.4736e-02,  ..., -5.6118e-03,
         -1.2590e-02, -2.8085e-02],
        [ 2.6413e-03, -2.3298e-02,  5.4922e-03,  ...,  1.7537e-02,
          2.7550e-02, -7.7656e-02]], device='cuda:0', requires_grad=True)

### Datasets

In [10]:
from datasets import load_dataset
# dataset = load_dataset('trec', split='test', cache_dir="./cache") #
raw_datasets = load_dataset('glue', 'sst2', cache_dir="./cache") #
# dataset = load_dataset('ag_news', split='test', cache_dir="./cache") #
# dataset = load_dataset('emotion', split='test', cache_dir="./cache") #
# raw_datasets = load_dataset("imdb", cache_dir="./cache/dataset", download_mode="force_redownload")

Reusing dataset glue (./cache/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

In [12]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Loading cached processed dataset at ./cache/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-7d5df1b353f66d1e.arrow
Loading cached processed dataset at ./cache/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-1386eaba7c00c412.arrow
Loading cached processed dataset at ./cache/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-dcd8bec60fadff5f.arrow


In [13]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(500))

Loading cached shuffled indices for dataset at ./cache/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d4bbf9acd20962c2.arrow
Loading cached shuffled indices for dataset at ./cache/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-ce7874c9ff1fa516.arrow


In [14]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [15]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [16]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [17]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

MyBertForSequenceClassification(
  (bert): BertModel(
    (embeddings): MyBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

In [18]:
def get_pe_variance(model):
    return float(torch.sum(torch.std(model.bert.embeddings.position_embeddings.weight, dim=1)))

In [19]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch, output_hidden_states=True)
#         if epoch % 50 == 0:
#             print(model.bert.embeddings.position_embeddings.weight[:5, :5])
        loss = outputs.loss
#         breg_penalty = self.regularizer(representation,
#                                         model,
#                                         disable_gradient=False,
#                                         **input_kwargs
#                                         )
#         print(model.bert.embeddings.position_embeddings_output[:3,:3])
        penalty = 1e-6*torch.norm(model.bert.embeddings.position_embeddings_output)
        print(float(penalty))
        loss += penalty
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/375 [00:00<?, ?it/s]

1.0076283615489956e-05
1.0076283615489956e-05
1.0076270882564131e-05
1.0076252692670096e-05
1.0076232683786657e-05
1.0076222679344937e-05
1.0076233593281358e-05
1.0076245416712482e-05
1.0076249054691289e-05
1.0076255421154201e-05
1.0076266335090622e-05
1.0076268154080026e-05
1.0076264516101219e-05
1.0076264516101219e-05
1.007626542559592e-05
1.0076264516101219e-05
1.0076251783175394e-05
1.0076237231260166e-05
1.0076218131871428e-05
1.0076206308440305e-05
1.0076194485009182e-05
1.007619357551448e-05
1.0076194485009182e-05
1.0076185390062165e-05
1.007617811410455e-05
1.007617811410455e-05
1.007617811410455e-05
1.0076176295115147e-05
1.007616810966283e-05
1.0076145372295287e-05
1.0076129910885356e-05
1.0076105354528408e-05
1.0076082617160864e-05
1.0076056241814513e-05
1.0076031685457565e-05
1.0076000762637705e-05
1.007597256830195e-05
1.0075953468913212e-05
1.007594164548209e-05
1.0075922546093352e-05
1.007590071822051e-05
1.0075871614390053e-05
1.0075844329549e-05
1.0075819773192052e-05


KeyboardInterrupt: 

In [ ]:
import sys
from collections import OrderedDict

sys.path.append("../")
from regularizers import JpRegularizer
from util.constants import TC_ExperimentData, TC_ModelType, Constants, MaxSenLen, RegRepresentation, Regularization, InputType, ModelType, SentenceOrdering

In [ ]:
regularizer = JpRegularizer(Regularization.edge, InputType.set, total_permutations=None,
                            strength=0.01, eps=0.1,
                            representations=RegRepresentation.pred, normed=False,
                            power=2, tangent_prop=True, tp_config=OrderedDict([("tp_num_tangent_vectors",1)]),
                            fixed_edge=False, random_segment=True,
                            permute_positional_encoding=True,
                            task_type=None,  # if cfg.task_type == TaskType.node_classification
                            agg_penalty_by_node=False)

In [ ]:
from datasets import load_metric
metric = load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()